## Covert to YOLO Labels

In [8]:
import json
from typing import Dict, Tuple
import os

lbl_file = "../../data/bdd100k/labels/100k/det_20/det_train.json"
output_dir = "train"
image_w = 1280
image_h = 720

In [13]:
# Define the category mapping
category_mapping = {
    "pedestrian": 0,
    "other person": 0,
    "rider": 1,
    "car": 2,
    "other vehicle": 2,  # alias of car
    "truck": 3,
    "trailer": 3,
    "bus": 4,
    "train": 5,
    "motorcycle": 6,
    "bicycle": 7,
    "traffic light": 8,
    "traffic sign": 9,
}

def convert_to_yolo(label, image_width, image_height):
    category = label["category"]
    category_id = category_mapping.get(category.lower(), -1)

    if category_id == -1:
        raise ValueError(f"Unknown category: {category}")

    x1 = label["box2d"]["x1"] / image_width
    y1 = label["box2d"]["y1"] / image_height
    x2 = label["box2d"]["x2"] / image_width
    y2 = label["box2d"]["y2"] / image_height

    # Calculate center coordinates and normalized width and height
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    width = x2 - x1
    height = y2 - y1

    return f"{category_id} {center_x} {center_y} {width} {height}\n"

def json_to_yolo(json_data, output_dir, img_w, img_h):
    os.makedirs(output_dir, exist_ok=True)
    image_width = img_w
    image_height = img_h

    for image_info in json_data:
        image_filename = image_info["name"]

        yolo_filename = os.path.join(output_dir, os.path.splitext(image_filename)[0] + ".txt")

        with open(yolo_filename, "w") as yolo_file:
            if "labels" in image_info:
                for label in image_info["labels"]:
                    try:
                        yolo_line = convert_to_yolo(label, image_width, image_height)
                        yolo_file.write(yolo_line)
                    except ValueError as e:
                        print(e.args[0])
                        continue
            else:
                yolo_file.write("")
    

In [14]:
with open(lbl_file, "r") as f:
    data = json.load(f)

json_to_yolo(data, output_dir, image_w, image_h)



## Build condition-specific dataset

In [2]:
from lib.data import tools
from lib.simulation.env import get_image_paths
from lib.data.condparser import ConditionParserMode
import os
import shutil

pkg_name = "100k"
train_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_train.json"
val_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_val.json"
image_path_train = "/home/zekun/drivable/tmp/dataset/images/train"
image_path_val = "/home/zekun/drivable/tmp/dataset/images/val"
img_train_path = "/home/zekun/drivable/tmp/100k_yolo/images/train"
img_val_path = "/home/zekun/drivable/tmp/100k_yolo/images/val"
lbl_train_path = "/home/zekun/drivable/tmp/100k_yolo/labels/train"
lbl_val_path = "/home/zekun/drivable/tmp/100k_yolo/labels/val"

def cond_list_to_fname(l):
    return '-'.join(l).replace(" ", "_").replace("/", "_or_")

def cond_fname_to_list(s:str):
    return s.replace("_or_", "/").replace("_", " ").split('-')

/home/zekun/drivable/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zekun/drivable/.venv/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/zekun/drivable/.venv/lib/python3.10/site-packages/torchvisio

In [17]:
all_conditions = tools.get_all_appeared_conditions(val_attr_file, ConditionParserMode.VALUE_LIST)

cnt = [0, 0]
for condition in all_conditions:
    train_images = tools.get_img_paths_by_conditions([condition], train_attr_file, image_path_train)
    val_images = tools.get_img_paths_by_conditions([condition], val_attr_file, image_path_val)
    l1, l2 = len(train_images), len(val_images)
    # if l1<=5 or l2<=5:
    #     continue
    fname = cond_list_to_fname(condition)
    print(f"{fname}:\t\t\t\t Train {l1}\t Val {l2}")
    cond_img_train_path = os.path.join(img_train_path, fname)
    cond_img_val_path = os.path.join(img_val_path, fname)
    cond_lbl_train_path = os.path.join(lbl_train_path, fname)
    cond_lbl_val_path = os.path.join(lbl_val_path, fname)
    if not os.path.exists(cond_img_train_path):
        os.mkdir(cond_img_train_path)
    if not os.path.exists(cond_img_val_path):
        os.mkdir(cond_img_val_path)
    if not os.path.exists(cond_lbl_train_path):
        os.mkdir(cond_lbl_train_path)
    if not os.path.exists(cond_lbl_val_path):
        os.mkdir(cond_lbl_val_path)
    for img in train_images:
        shutil.copy(img, cond_img_train_path)
        shutil.copy(img.replace("images", "labels").replace(".jpg", ".txt"), cond_lbl_train_path)
    for img in val_images:
        shutil.copy(img, cond_img_val_path)
        shutil.copy(img.replace("images", "labels").replace(".jpg", ".txt"), cond_lbl_val_path)
    cnt[0] += l1
    cnt[1] += l2
print(cnt)

partly_cloudy-parking_lot-daytime:				 Train 31	 Val 4
overcast-parking_lot-daytime:				 Train 56	 Val 6
undefined-city_street-undefined:				 Train 6	 Val 2
clear-tunnel-night:				 Train 6	 Val 1
snowy-residential-night:				 Train 230	 Val 39
undefined-residential-night:				 Train 25	 Val 4
rainy-residential-dawn_or_dusk:				 Train 41	 Val 4
overcast-city_street-dawn_or_dusk:				 Train 673	 Val 109
rainy-undefined-undefined:				 Train 4	 Val 1
partly_cloudy-undefined-daytime:				 Train 0	 Val 1
rainy-gas_stations-daytime:				 Train 2	 Val 1
snowy-highway-daytime:				 Train 444	 Val 70
clear-gas_stations-night:				 Train 7	 Val 2
clear-undefined-dawn_or_dusk:				 Train 1	 Val 1
snowy-residential-dawn_or_dusk:				 Train 75	 Val 16
snowy-parking_lot-night:				 Train 9	 Val 2
snowy-highway-dawn_or_dusk:				 Train 46	 Val 9
overcast-residential-daytime:				 Train 1114	 Val 154
undefined-undefined-daytime:				 Train 28	 Val 3
rainy-city_street-night:				 Train 1555	 Val 196
foggy-city_street

## generate yaml file

In [7]:
import yaml

all_conditions = tools.get_all_appeared_conditions(val_attr_file, ConditionParserMode.VALUE_LIST)

subdataset = [
    ['clear', 'city street', 'dawn/dusk'],
    ['partly cloudy', 'highway', 'dawn/dusk'],
    ['partly cloudy', 'residential', 'dawn/dusk'],
    ['overcast', 'city street', 'dawn/dusk'],
    ['rainy', 'highway', 'daytime'],
    ['snowy', 'city street', 'dawn/dusk'],
    ['rainy', 'residential', 'daytime'],
    ['partly cloudy', 'residential', 'dawn/dusk'],
    ['partly cloudy', 'highway', 'dawn/dusk'],
    ['partly cloudy', 'city street', 'dawn/dusk'],
    ['clear', 'parking lot', 'daytime']
]
# Specify the path for the YAML file
yaml_file_path = 'cfg/other.yaml'

train_lst = []
val_lst = []
for condition in all_conditions:
    # if "snowy" not in condition and "rainy" not in condition:
    if condition in subdataset:
        print("Add ", condition)
        train_lst.append(os.path.join(img_train_path, cond_list_to_fname(condition)))
        val_lst.append(os.path.join(img_val_path, cond_list_to_fname(condition)))

print("Condition Number:", len(train_lst))

# Data to be written to the YAML file
data = {
    "path": "/home/zekun/drivable/tmp/dataset",
    "train": train_lst,
    "val": val_lst,
    'names': {
        0: 'pedestrian',
        1: 'rider',
        2: 'car',
        3: 'truck',
        4: 'bus',
        5: 'train',
        6: 'motorcycle',
        7: 'bicycle',
        8: 'traffic light',
        9: 'traffic sign'
    }
}

# Write data to the YAML file
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(data, yaml_file, default_flow_style=False)

print(f"YAML file '{yaml_file_path}' created successfully.")

Add  ['partly cloudy', 'city street', 'dawn/dusk']
Add  ['clear', 'city street', 'dawn/dusk']
Add  ['clear', 'parking lot', 'daytime']
Add  ['partly cloudy', 'highway', 'dawn/dusk']
Add  ['rainy', 'residential', 'daytime']
Add  ['partly cloudy', 'residential', 'dawn/dusk']
Add  ['rainy', 'highway', 'daytime']
Add  ['overcast', 'city street', 'dawn/dusk']
Add  ['snowy', 'city street', 'dawn/dusk']
Condition Number: 9
YAML file 'cfg/other.yaml' created successfully.
